
# Lyrics Finder Tool using OpenAGI and Serper API

This notebook demonstrates how to create a Custom Lyrics Finder Tool using OpenAGI and the Serper API. The tool searches for song information based on the artist and song title.

## Setup Instructions

1. **Restart the Colab session**: To avoid dependency errors, please restart the Colab session before running the code.

   - Go to Runtime > Restart runtime
   - After restarting, run all cells from the beginning

2. **Install required packages**:
   Run the following cell to install the necessary packages:

In [ ]:
pip install openagi langchain-google-genai

Importing necessary modules

In [21]:
import os
import json
import http.client
from typing import Any
from openagi.actions.base import BaseAction
from pydantic import Field
from openagi.agent import Admin
from openagi.llms.openai import OpenAIModel
from openagi.memory import Memory
from openagi.planner.task_decomposer import TaskPlanner
from openagi.worker import Worker
from openagi.actions.tools.serper_search import SerperSearch

 **Set up API keys**:
You'll need to set up the OpenAI API key

In [22]:
os.environ['OPENAI_API_KEY'] = "sk-<your-api-key>"

config = OpenAIModel.load_from_env_config()
llm = OpenAIModel(config=config)

### LyricsFinderTool Class:
- Custom tool utilizing Serper API for song information searches
- Implements the execute method for performing searches


In [30]:

class LyricsFinderTool(BaseAction):
    """
    This tool searches for song lyrics based on the artist and song title using Serper.
    """
    artist: str = Field(
        default_factory=str,
        description="Name of the artist whose lyrics are to be searched."
    )
    song_title: str = Field(
        default_factory=str,
        description="Title of the song whose lyrics are to be searched."
    )

    def execute(self):
        # Construct the search query
        query = f"{self.artist} {self.song_title} lyrics"

        # Set up the connection to Serper API
        conn = http.client.HTTPSConnection("google.serper.dev")
        payload = json.dumps({"q": query})
        headers = {
            'X-API-KEY': '<your-serper-api-key>',
            'Content-Type': 'application/json'
        }

        # Make the request
        conn.request("POST", "/search", payload, headers)
        res = conn.getresponse()
        data = json.loads(res.read().decode("utf-8"))

        # Extract lyrics from the search results
        full_lyrics = ""
        for result in data.get('organic', []):
            if 'snippet' in result:
                full_lyrics += result['snippet'] + "\n\n"

        if full_lyrics:
            return full_lyrics.strip()
        else:
            return "Sorry, no lyrics found for the given artist and song title."

### Worker Setup:
- Creates a specialized worker for song information retrieval
- Defines the role and instructions for the worker

In [33]:
# Create a worker for searching lyrics
song_searcher = Worker(
    role="Lyrics Searcher",
    instructions="""
    You are an Expert Music Search Assistant with a deep understanding of song lyrics and music databases.
    - Focus on finding lyrics for specific songs based on user queries.
    - Utilize the Lyrics Finder Tool to search for song lyrics by song name and optional artist name.
    - Ensure accurate retrieval of lyrics and provide them in a user-friendly format.
    - Be attentive to user preferences and provide additional context about the song when available.
    """,
    actions=[LyricsFinderTool],
)

### Admin Setup:
- Configures an Admin to manage the worker and execute tasks
- Utilizes OpenAI's language model for natural language processing

In [34]:
admin = Admin(
    actions=[],
    planner=TaskPlanner(human_intervene=False),
    llm=llm,
)

admin.assign_workers([song_searcher])


### Execution:
- Runs the admin with a specific query to find song information

In [37]:
res = admin.run(
    query="provide the lyrics of the song",
    description="""
    Please provide the lyrics for the song "Bohemian Rhapsody"( 1975) by the band Queen. The lyrics should be formatted in a clear and easy-to-read manner, with each verse and chorus clearly separated.
    """,
)
print(res)




Tasks: [{'task_name': "Retrieve Lyrics for 'Bohemian Rhapsody' by Queen", 'description': "Use the LyricsFinderTool to search and retrieve the lyrics for the song 'Bohemian Rhapsody' by the band Queen. The lyrics should be formatted in a clear and easy-to-read manner, with each verse and chorus clearly separated. Provide the lyrics in a user-friendly format and ensure the accuracy of the content.", 'worker_id': '5d3a5fde7ab84b299c079fc9c15aa26b'}]


### Final Output

[Intro]  
Is this the real life? Is this just fantasy?  
Caught in a landslide, no escape from reality.  
Open your eyes, look up to the skies and see  

[Guitar Solo 2]  
Ooh, yeah. Ooh, yeah  

[Outro: Freddie Mercury & (Audience)]  
Nothing really matters, anyone can see (Nothing really matters)  

Is this the real life? Is this just fantasy?  
Caught in a landslide. No escape from reality.  
Open your eyes. Look up to the skies and see  

I'm just a poor boy and nobody loves me.  
He's just a poor boy from a poor fami